In [ ]:
from SALib.analyze import sobol
from ema_workbench import Samplers
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench import (
    TimeSeriesOutcome, 
    ScalarOutcome,
    perform_experiments, 
    RealParameter,
    CategoricalParameter,
    ema_logging, 
    MultiprocessingEvaluator,
    ScalarOutcome,
    ArrayOutcome,
    Constant,
    Model,
    Policy
)
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
wd = r"./Models"
model = VensimModel("simpleModel", wd=wd, model_file="Thesismodel no policy1.vpmx") 

In [ ]:
model.outcomes = [
    TimeSeriesOutcome('TIME'),
    TimeSeriesOutcome("MW installed"),
    TimeSeriesOutcome("LCOH[green hydrogen]"),
    #TimeSeriesOutcome("total project funnel[green hydrogen]"),
    #TimeSeriesOutcome("Hydrogen type demand[green hydrogen]"),
    #TimeSeriesOutcome("total development time[green hydrogen]"),
    TimeSeriesOutcome("satisfied demand")
]

In [ ]:
model.uncertainties = [
    RealParameter("Demand substitution rate",0.78,2),
    RealParameter("Roadmap WOZ",1000,1500),
    RealParameter("Efficiency increase",0.01,0.017),
]

In [ ]:
sa_results

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from SALib.analyze import sobol
from ema_workbench import Samplers
#from ema_workbench.em_framework.evaluators import SOBOL
from ema_workbench.em_framework import get_SALib_problem
n_exp = 200
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    sa_results = evaluator.perform_experiments(scenarios=n_exp, uncertainty_sampling=Samplers.SOBOL)

In [ ]:
experiments, outcomes = sa_results
outcomes

In [ ]:
from SALib.analyze.sobol import analyze

problem = get_SALib_problem(model.uncertainties)
final_sobol = outcomes['MW installed'][:,-1]
mean_sobol = np.mean(outcomes['MW installed'][:,:],axis=1)
std_sobol = np.std(outcomes['MW installed'][:,:],axis=1)

Si = sobol.analyze(problem, final_sobol, calc_second_order=True, print_to_console=True)


In [ ]:
Si_filter = {k:Si[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

sns.set_style('white')
fig, ax = plt.subplots(1)

indices = Si_df[['S1','ST']]
err = Si_df[['S1_conf','ST_conf']]

indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(8,6)
fig.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
Y = mean_sobol

s_data = pd.DataFrame(index=problem['names'],
                      columns=np.arange(20,n_exp,50)*(2*problem['num_vars']+2))
for j in s_data.columns:
    scores = sobol.analyze(problem, Y[0:j], calc_second_order=True, print_to_console=False)
    s_data.loc[:,j] = scores['ST']

In [ ]:
fig, ax = plt.subplots(1)

s_data.T.plot(ax=ax)
ax.set_xlabel('Samples')
ax.set_ylabel('Total index (ST)')
plt.show()

In [ ]:
sobol_results = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    y = final_sobol[logical]
    indices = analyze(problem, y)
    sobol_results[policy] = indices

In [ ]:
indices

In [ ]:
plt.hist(y)
plt.show()

In [ ]:
from ema_workbench.analysis import feature_scoring

In [ ]:
import seaborn as sns

for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    subset_results = {k:v[logical] for k,v in outcomes.items()}
    scores = feature_scoring.get_feature_scores_all(experiments[logical],
                                                   subset_results)
    sns.heatmap(scores, annot=True, cmap='viridis')
    plt.show()